# RMB Fly Behavior Analysis

This notebook performs comprehensive analysis of fly behavior data using R functions through rpy2. The analysis includes:

- Data loading and preprocessing
- Sleep analysis and pattern detection
- Position tracking and movement analysis
- Visualization with heatmaps and bar plots
- Time-series analysis across multiple days

**Author:** Yongjun Li  
**Date:** 2024-07-17  
**Analysis Date:** 20250717

In [1]:
# Setup R Environment and Load Libraries
import subprocess
import sys

# Install required Python packages if missing (suppress output)
required_packages = ['numpy', 'pandas', 'rpy2']
for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package], 
                            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Now import the packages
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import pandas as pd
import numpy as np
import os

# Get rpy2 version safely
try:
    rpy2_version = rpy2.__version__
except AttributeError:
    try:
        import rpy2._version
        rpy2_version = rpy2._version.version
    except:
        rpy2_version = "unknown"

print(f"rpy2 version: {rpy2_version}")

# Load essential R libraries (suppress messages) - only run once
if 'r_libraries_loaded' not in globals():
    r('''
    # Suppress package startup messages and conflicts
    suppressPackageStartupMessages({
      # Install required packages if not already installed
      if (!requireNamespace("ComplexHeatmap", quietly = TRUE)) {
          if (!requireNamespace("BiocManager", quietly = TRUE))
              install.packages("BiocManager", repos="https://cran.rstudio.com/", quiet=TRUE)
          BiocManager::install("ComplexHeatmap", ask=FALSE, update=FALSE)
      }

      required_packages <- c("tidyverse", "ggplot2", "dplyr", "tidyr", 
                            "circlize", "lubridate", "rstudioapi")

      for (pkg in required_packages) {
          if (!requireNamespace(pkg, quietly = TRUE)) {
              install.packages(pkg, repos="https://cran.rstudio.com/", quiet=TRUE)
          }
      }

      # Load libraries with suppressed messages
      library(ComplexHeatmap)
      library(circlize)
      library(tidyverse)
      library(ggplot2)
      library(dplyr)
      library(tidyr)
      library(lubridate)
    })

    cat("✓ R libraries loaded successfully\\n")
    ''')
    r_libraries_loaded = True
else:
    print("✓ R libraries already loaded (skipping re-initialization)")

print("✓ Setup completed!")

/home/liy27/miniconda3/envs/rmb-analysis/lib/python3.13/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/home/liy27", R: "/home/liy27/projects/RMB"
  warnings.warn(
/home/liy27/miniconda3/envs/rmb-analysis/lib/python3.13/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/tmp/RtmpHrQNap", R: "/tmp/RtmpGE3bYH"
  warnings.warn(
/home/liy27/miniconda3/envs/rmb-analysis/lib/python3.13/site-packages/rpy2/rinterface/__init__.py:1211: UserWarning: Environment variable "R_SESSION_TMPDIR" redefined by R and overriding existing variable. Current: "/tmp/RtmpHrQNap", R: "/tmp/RtmpGE3bYH"
  warnings.warn(


rpy2 version: unknown
✓ R libraries loaded successfully
✓ Setup completed!
✓ R libraries loaded successfully
✓ Setup completed!


In [2]:
# Kernel Health Check and Memory Optimization
import gc
import os
import psutil
import sys

print("=== KERNEL HEALTH CHECK ===")

# Check Python environment
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Check memory usage
process = psutil.Process()
memory_info = process.memory_info()
memory_mb = memory_info.rss / 1024 / 1024
print(f"Current memory usage: {memory_mb:.1f} MB")

# Check available memory
available_memory = psutil.virtual_memory().available / 1024 / 1024 / 1024
print(f"Available system memory: {available_memory:.1f} GB")

# Force Python garbage collection
collected = gc.collect()
print(f"Python garbage collection freed {collected} objects")

# Check rpy2 availability
try:
    import rpy2
    print(f"✓ rpy2 available")
except ImportError as e:
    print(f"✗ rpy2 import error: {e}")

# Check working directory
print(f"Current working directory: {os.getcwd()}")
print(f"RMB project exists: {os.path.exists('/home/liy27/projects/RMB')}")

print("=== HEALTH CHECK COMPLETE ===")
print("If all checks pass, you can proceed with the analysis.")
print("If memory usage is >1GB, consider restarting the kernel.")

=== KERNEL HEALTH CHECK ===
Python version: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:27:50) [GCC 13.3.0]
Python executable: /home/liy27/miniconda3/envs/rmb-analysis/bin/python
Current memory usage: 361.0 MB
Available system memory: 11.0 GB
Python garbage collection freed 45 objects
✓ rpy2 available
Current working directory: /home/liy27/projects/RMB
RMB project exists: True
=== HEALTH CHECK COMPLETE ===
If all checks pass, you can proceed with the analysis.
If memory usage is >1GB, consider restarting the kernel.


In [3]:
# Configure Project Directories and Parameters
r('''
# Load the config and modify here as needed
base_dir <- "/home/liy27/projects/RMB"  # change this to where you own RMB path
setwd(base_dir)
data_dir <- "./test/"  # change this to where your data is
date <- "20250717"

cat("Base directory:", base_dir, "\\n")
cat("Data directory:", data_dir, "\\n")
cat("Analysis date:", date, "\\n")
cat("Current working directory:", getwd(), "\\n")
''')

Base directory: /home/liy27/projects/RMB 
Data directory: ./test/ 
Analysis date: 20250717 
 /home/liy27/projects/RMB 
Data directory: ./test/ 
Analysis date: 20250717 
Current working directory: /home/liy27/projects/RMB 
Current working directory: /home/liy27/projects/RMB 


In [4]:
# Load and Source R Functions
r('''
base_dir <- "/home/liy27/projects/RMB"
setwd(base_dir)

# Source only the essential scripts initially
source("./src/metadata.R")
source("./src/metadata_workflow.R")

# Source essential processing scripts
source("./src/process_monitorS4_data.R")
source("./src/data_transformations.R")

# Try to source additional scripts with error handling
tryCatch({
  source("./src/experiment_processing.R") 
}, error = function(e) {
  cat("Note: experiment_processing.R not loaded:", e$message, "\\n")
})

tryCatch({
  source("./src/plotting_utilities.R")
}, error = function(e) {
  cat("Note: plotting_utilities.R not loaded:", e$message, "\\n")
})

cat("Essential R functions loaded successfully!\\n")
''')

Essential R functions loaded successfully!


In [5]:
# Check if metadata already exists and skip generation if present
r('''
metadata_dir <- file.path(data_dir, "metadata")
metadata_exists <- dir.exists(metadata_dir) && length(list.files(metadata_dir, pattern = "_metadata.csv")) > 0

if (metadata_exists) {
    cat("✓ Metadata files already exist in", metadata_dir, "\\n")
    cat("Skipping metadata generation step.\\n")
    
    # List existing metadata files
    existing_files <- list.files(metadata_dir, pattern = "_metadata.csv", full.names = FALSE)
    cat("Found existing metadata files:\\n")
    for (file in existing_files) {
        cat("  -", file, "\\n")
    }
} else {
    cat("⚠️  No metadata files found. Next cell will generate metadata.\\n")
}
''')

✓ Metadata files already exist in ./test//metadata 
Skipping metadata generation step.
Found existing metadata files:
  -  ./test//metadata 
Skipping metadata generation step.
Found existing metadata files:
  - Monitor36_metadata.csv Monitor36_metadata.csv 
  - Monitor37_metadata.csv 
  - Monitor38_metadata.csv 
  - Monitor39_metadata.csv 
  - Monitor40_metadata.csv 
  -
  - Monitor37_metadata.csv 
  - Monitor38_metadata.csv 
  - Monitor39_metadata.csv 
  - Monitor40_metadata.csv 
  - Monitor41_metadata.csv Monitor41_metadata.csv 
 


If you have your metadata file ready, you can run the analysis directly. The metadata file should be in CSV format with columns for monitor number, channel, group, treatment, sex, and genotype. You can also mannually edit the metadata file to include additional information such as lab, user, and date.The current script to generate metadata is `generate_metadata_test.R`, which creates a sample metadata file for testing purposes. Notice: the current code assumes that you load the same group flies for ever y 16 channels, so if your experiment has different setting for monitor, you need to manually edit the metadata file to match your experiment setup.

In [6]:
# Generate and Load Metadata
r('''
# Get the metadata directory path
metadata_dir <- file.path(data_dir, "metadata")

# Check if we need to generate metadata (this check was already done in previous cell)
metadata_exists <- dir.exists(metadata_dir) && length(list.files(metadata_dir, pattern = "_metadata.csv")) > 0

if (!metadata_exists) {
    # Generate metadata only if it doesn't exist
    cat("Generating metadata files...\\n")
    source("./generate_metadata_test.R")
    cat("\\nMetadata generation completed!\\n")
}

# Display detailed fly information for all monitors by channel ranges
cat("\\n=== MONITOR INFORMATION BY CHANNEL RANGES ===\\n")

# Read the source metadata CSV to show the channel range assignments
metadata_source <- read.csv("./test/test_metadata.csv", stringsAsFactors = FALSE)

# Group by monitor and display channel ranges
monitors <- unique(metadata_source$Monitor_number)
for (monitor in monitors) {
  monitor_data <- metadata_source[metadata_source$Monitor_number == monitor, ]
  
  cat("\\n🔬 ", monitor, " Configuration:\\n")
  cat("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\n")
  
  for (i in 1:nrow(monitor_data)) {
    row <- monitor_data[i, ]
    cat(sprintf("  Channels %2d-%2d: %s | %s | %s | %s | %s°C | %s\\n",
               row$Start_channel,
               row$End_channel,
               row$Group,
               row$Treatment,
               row$Sex,
               row$Genotype,
               row$Temperature,
               row$Incubator))
  }
}

cat("\\n=== DETAILED FLY INFORMATION BY INDIVIDUAL CHANNELS ===\\n")
if (dir.exists(metadata_dir)) {
  metadata_files <- list.files(metadata_dir, pattern = "_metadata.csv", full.names = TRUE)
  
  for (file in metadata_files) {
    metadata <- read.csv(file)
    monitor_name <- gsub("_metadata.csv", "", basename(file))
    
    cat("\\n📊 ", monitor_name, " - Individual Fly Information\\n")
    cat("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\n")
    
    # Display information for all 32 channels (assuming full monitor capacity)
    max_channel <- max(metadata$Channel, na.rm = TRUE)
    for (ch in 1:max_channel) {
      fly_info <- metadata[metadata$Channel == ch, ]
      if (nrow(fly_info) > 0) {
        cat(sprintf("Channel %2d: %s | %s | %s | %s | %s\\n",
                   ch,
                   fly_info$Group[1],
                   fly_info$Treatment[1], 
                   fly_info$Sex[1],
                   fly_info$Genotype[1],
                   ifelse(fly_info$Alive[1], "✓", "✗")))
      }
    }
    
    # Summary statistics for this monitor
    total_flies <- nrow(metadata)
    alive_flies <- sum(metadata$Alive, na.rm = TRUE)
    treatments <- unique(metadata$Treatment)
    groups <- unique(metadata$Group)
    
    cat("\\n📈 Summary for ", monitor_name, ":\\n")
    cat("   Total flies: ", total_flies, " | Alive: ", alive_flies, " | Dead: ", (total_flies - alive_flies), "\\n")
    cat("   Treatments: ", paste(treatments, collapse = ", "), "\\n")
    cat("   Groups: ", paste(groups, collapse = ", "), "\\n")
  }
} else {
  cat("⚠️  No metadata directory found at:", metadata_dir, "\\n")
}

cat("\\n✅ Detailed fly information display completed!\\n")
''')


=== MONITOR INFORMATION BY CHANNEL RANGES ===

🔬  Monitor36  Configuration:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Channels  1-16: Male_CS | Control | Male | Wcs | 25°C | Incubator1
  Channels 17-32: Male_CS | Control | Male | Wcs | 25°C | Incubator1

🔬  Monitor37  Configuration:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔬  Monitor36  Configuration:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Channels  1-16: Male_CS | Control | Male | Wcs | 25°C | Incubator1
  Channels 17-32: Male_CS | Control | Male | Wcs | 25°C | Incubator1

🔬  Monitor37  Configuration:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Channels  1-16: Female_CS | Control | Female | Wcs | 25°C | Incubator1
  Channels 17-32: Female_CS | Control | Female | Wcs | 25°C | Incubator1

🔬  Monitor38  Configuration:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Channels  1-16: Male_Gaboxdol | Gaboxdol | Male | Wcs 

In [7]:
# Load Monitor Data Files
r('''
# Set memory management options
options(max.print = 500)
gc()  # Force garbage collection before loading

# Get a list of all the .txt files in the directory
monitor_files <- load_monitor_files(data_dir)
cat("Found", length(monitor_files), "monitor files:\\n")

# Limit file listing to prevent output overflow
max_files_to_show <- min(10, length(monitor_files))
for (i in 1:max_files_to_show) {
  cat("  ", i, ":", basename(monitor_files[i]), "\\n")
}
if (length(monitor_files) > max_files_to_show) {
  cat("  ... (", length(monitor_files) - max_files_to_show, " more files)\\n")
}

# Check if create_monitorS4 function exists
if (!exists("create_monitorS4")) {
  stop("create_monitorS4 function not found even after sourcing process_monitorS4_data.R")
}

# Create a memory-efficient version of the function
create_monitorS4_quiet <- function(file_path) {
  tryCatch({
    if (file.exists(file_path)) {
      # Read the raw data with memory management
      raw_data <- read.table(file_path, header = FALSE, sep = "\\t", 
                            fill = TRUE, stringsAsFactors = FALSE,
                            nrows = -1)  # Let R determine optimal reading
      
      # Get monitor name from filename
      monitor_name <- tools::file_path_sans_ext(basename(file_path))
      
      # Create the monitor object structure
      monitor_obj <- list(
        file_path = file_path,
        monitor_name = monitor_name,
        raw_data = raw_data,
        assays = list(),
        meta.data = NULL
      )
      
      class(monitor_obj) <- "monitorS4"
      
      # Force garbage collection after each file
      gc()
      
      return(monitor_obj)
    } else {
      stop("File not found:", file_path)
    }
  }, error = function(e) {
    cat("Error loading", basename(file_path), ":", e$message, "\\n")
    return(NULL)
  })
}

# Create monitor list with memory management
cat("Creating monitor objects...\\n")
monitor_list <- list()

# Process files with memory checks
for (i in 1:length(monitor_files)) {
  cat("  Processing", i, "of", length(monitor_files), ":", basename(monitor_files[i]), "\\n")
  
  monitor_obj <- create_monitorS4_quiet(monitor_files[i])
  if (!is.null(monitor_obj)) {
    monitor_list[[i]] <- monitor_obj
    cat("  ✓ Created object for", basename(monitor_files[i]), "\\n")
  } else {
    cat("  ✗ Failed to create object for", basename(monitor_files[i]), "\\n")
  }
  
  # Memory management between files
  if (i %% 2 == 0) {  # Every 2 files
    gc()
  }
}

# Remove any NULL entries
monitor_list <- monitor_list[!sapply(monitor_list, is.null)]

cat("✓ Created monitor_list with", length(monitor_list), "elements\\n")

# Final memory cleanup
gc()
''')

Found 6 monitor files:
   1  6 monitor files:
   1 :: Monitor36.txt 
   2 : Monitor37.txt 
   Monitor36.txt 
   2 : Monitor37.txt 
   3 : Monitor38.txt 
    3 : Monitor38.txt 
   4 : Monitor39.txt 
   5 : Monitor40.txt 
   6 : Monitor41.txt4 : Monitor39.txt 
   5 : Monitor40.txt 
   6 : Monitor41.txt 
Creating monitor objects...
 
Creating monitor objects...
  Processing 1  Processing 1 of 6 : Monitor36.txt  of 6 : Monitor36.txt 

  ✓ Created object for Monitor36.txt 
  Processing 2 of 6 : Monitor37.txt  ✓ Created object for Monitor36.txt 
  Processing 2 of 6 : Monitor37.txt 
 
  ✓ Created object for Monitor37.txt 
  ✓ Created object for Monitor37.txt 
  Processing 3 of 6 : Monitor38.txt 
  Processing 3 of 6 : Monitor38.txt 
  ✓ Created object for Monitor38.txt 
  Processing 4 of 6 : Monitor39.txt 
  ✓ Created object for Monitor38.txt 
  Processing 4 of 6 : Monitor39.txt 
  ✓ Created object for Monitor39.txt 
  ✓ Created object for Monitor39.txt 
  Processing 5 of 6 : Monitor40.txt 
  

2464694.000000,7533679.000000,131.700000,...,11243618.000000,189.000000,85.800000


In [8]:
# Process Monitor Data
r('''
# Load metadata files
metadata_files <- list.files(path = file.path(data_dir, "metadata/"),
                            pattern = "metadata", full.names = TRUE)

if (length(metadata_files) > 0) {
  cat("Found", length(metadata_files), "metadata files\\n")
  
  for (i in 1:min(length(metadata_files), length(monitor_list))) {
    metadata <- read.csv(metadata_files[i])
    monitor_list[[i]]$meta.data <- metadata
    cat("Loaded metadata for monitor", i, "- dimensions:", dim(metadata), "\\n")
  }
  
  if (length(monitor_list) > 0 && !is.null(monitor_list[[1]]$meta.data)) {
    cat("Sample metadata columns:", paste(colnames(monitor_list[[1]]$meta.data), collapse = ", "), "\\n")
  }
} else {
  cat("Warning: No metadata files found in", file.path(data_dir, "metadata/"), "\\n")
}

# Process the data
if (file.exists("./src/process_monitorS4_data.R")) {
  source("./src/process_monitorS4_data.R")
  for (i in 1:length(monitor_list)) {
    monitor_list[[i]] <- process_monitorS4_data(monitor_list[[i]])
  }
  cat("Data processing completed for all monitors\\n")
} else {
  cat("Warning: process_monitorS4_data.R not found. Skipping data processing step.\\n")
}
''')

Found 6 metadata files
 6 metadata files
Loaded metadata for monitor 1 - dimensions: 32 16 
Loaded metadata for monitor 2 Loaded metadata for monitor 1 - dimensions: 32 16 
Loaded metadata for monitor 2 - dimensions: 32 16 
Loaded metadata for monitor 3 - dimensions: 32 16 
Loaded metadata for monitor- dimensions: 32 16 
Loaded metadata for monitor 3 - dimensions: 32 16 
Loaded metadata for monitor 4 - dimensions: 32 16 
Loaded metadata for monitor  4 - dimensions: 32 16 
Loaded metadata for monitor 5 - dimensions: 32 16 
Loaded metadata for monitor 65 - dimensions: 32 16 
Loaded metadata for monitor 6 - dimensions: 32 16 
Sample metadata columns: Fly, Lab, User, Date, Experiment_name, Monitor_type, Monitor_number, Channel, Group, Tube_type, Incubator, Temperature, Treatment, Sex, Other, Alive 
 - dimensions: 32 16 
Sample metadata columns: Fly, Lab, User, Date, Experiment_name, Monitor_type, Monitor_number, Channel, Group, Tube_type, Incubator, Temperature, Treatment, Sex, Other, Aliv

In [9]:
# Display Loaded Fly Information by Channel
r('''
# Clear any existing output buffers and set memory limits
options(max.print = 1000)
gc()  # Force garbage collection

cat("\\n=== LOADED FLY INFORMATION BY MONITOR AND CHANNEL ===\\n")

if (length(monitor_list) > 0) {
  for (i in seq_along(monitor_list)) {
    # Add error handling and memory management
    tryCatch({
      if (!is.null(monitor_list[[i]]$meta.data)) {
        metadata <- monitor_list[[i]]$meta.data
        
        # Ensure we have valid data before processing
        if (nrow(metadata) > 0 && "Monitor_number" %in% colnames(metadata)) {
          monitor_name <- unique(metadata$Monitor_number)[1]
          
          cat("\\n🔬 Monitor", i, "(", monitor_name, ") - Channel Assignment:\\n")
          cat("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\\n")
          
          # Limit channel display to avoid memory issues
          max_channels <- min(32, max(metadata$Channel, na.rm = TRUE))
          channels_displayed <- 0
          
          for (ch in 1:max_channels) {
            fly_info <- metadata[metadata$Channel == ch, ]
            if (nrow(fly_info) > 0) {
              cat(sprintf("  Channel %2d: %s | %s | %s | %s | %s\\n",
                         ch,
                         fly_info$Group[1],
                         fly_info$Treatment[1], 
                         fly_info$Sex[1],
                         fly_info$Genotype[1],
                         ifelse(fly_info$Alive[1], "✓", "✗")))
              channels_displayed <- channels_displayed + 1
              
              # Prevent excessive output that could crash kernel
              if (channels_displayed >= 16) {
                if (max_channels > 16) {
                  cat("  ... (", (max_channels - 16), " more channels)\\n")
                }
                break
              }
            }
          }
          
          # Summary statistics
          total_channels <- nrow(metadata)
          alive_count <- sum(metadata$Alive, na.rm = TRUE)
          dead_count <- total_channels - alive_count
          
          cat("\\n  📊 Summary: ", total_channels, " total | ", alive_count, " alive | ", dead_count, " dead\\n")
          
          # Group breakdown with limit
          group_counts <- table(metadata$Group)
          cat("  📋 Groups: ")
          group_names <- names(group_counts)
          for (j in seq_along(group_names)) {
            if (j <= 3) {  # Limit to 3 groups to prevent overflow
              cat(group_names[j], "(", group_counts[group_names[j]], ") ")
            } else if (j == 4) {
              cat("... (", length(group_names) - 3, " more groups)")
              break
            }
          }
          cat("\\n")
          
          # Force flush output and garbage collection between monitors
          flush.console()
          gc()
        } else {
          cat("\\n⚠️  Monitor", i, "- Invalid metadata structure\\n")
        }
      } else {
        cat("\\n⚠️  Monitor", i, "- No metadata loaded\\n")
      }
    }, error = function(e) {
      cat("\\n❌ Error processing Monitor", i, ":", e$message, "\\n")
    })
  }
} else {
  cat("\\n⚠️  No monitor data loaded\\n")
}

cat("\\n✅ Fly information display completed!\\n")
gc()  # Final garbage collection
''')


=== LOADED FLY INFORMATION BY MONITOR AND CHANNEL ===

🔬 Monitor 1 ( Monitor36 ) - Channel Assignment:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ... ( 16  more channels)

  📊 Summary:  
🔬 Monitor 1 ( Monitor36 ) - Channel Assignment:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ... ( 16  more channels)

  📊 Summary:  32  total |  0  alive |  32  dead
  📋 Groups: Male_CS ( 32 ) 32  total |  0  alive |  32  dead
  📋 Groups: Male_CS ( 32 ) 


🔬 Monitor 2 ( Monitor37 ) - Channel Assignment:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ... (
🔬 Monitor 2 ( Monitor37 ) - Channel Assignment:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  ... ( 16  more channels)

  📊 Summary:  32  total |  0  alive |  32 16  more channels)

  📊 Summary:  32  total |  0  alive |  32  dead
  📋 Groups:   dead
  📋 Groups: Female_CS ( 32 ) 
Female_CS ( 32 ) 

🔬 Monitor 3 ( Monitor38 ) - Channel Assignment:
━━━━━━━━━━━━━━━━━━━━

2466683.000000,7540787.000000,131.800000,...,11243618.000000,189.000000,85.800000


In [ ]:
# Select Time Range and Merge Data
r('''
# Select time range
if (file.exists("./src/select_time_range.R")) {
  source("./src/select_time_range.R")
  monitor_list <- select_time_range(monitor_list)
  cat("Time range selection completed\\n")
} else {
  cat("Warning: select_time_range.R not found. Skipping time range selection.\\n")
}

# Merge the data
if (file.exists("./src/mergeMonitorS4.R")) {
  source("./src/mergeMonitorS4.R")
  monitor <- mergeMonitorS4(monitor_list)
  cat("Data merging completed successfully\\n")
  cat("Monitor object structure:\\n")
  cat("- Assays available:", names(monitor$assays), "\\n")
  cat("- Metadata dimensions:", dim(monitor$meta.data), "\\n")
} else {
  cat("Warning: mergeMonitorS4.R not found. Cannot proceed with data merging.\\n")
  cat("This is required for the visualization steps.\\n")
  stop("Missing required mergeMonitorS4.R function")
}
''')

Selecting time range for 6 monitors
Time range selection completed
 6 monitors
Time range selection completed
Merging 6 monitors into single object
Merging 6 monitors into single object


In [ ]:
# Remove Dead Flies
r('''
# Get rid of dead flies
if (file.exists("./src/remove_dead_flies.R")) {
  source("./src/remove_dead_flies.R")
  monitor <- remove_dead_flies(monitor)
  cat("Dead fly removal completed\\n")
  cat("Final data dimensions:\\n")
  cat("- Movement data (mt):", dim(monitor$assays$mt), "\\n")
  cat("- Position data (pn):", dim(monitor$assays$pn), "\\n")
} else {
  cat("Warning: remove_dead_flies.R not found. Skipping dead fly removal.\\n")
}
''')

In [ ]:
# Create Movement (mt) Heatmap
r('''
# Heatmap of movement data (mt)
dat <- as.matrix(monitor$assays$mt)
colnames(dat) <- colnames(monitor$assays$mt)

# Create a factor that divides the rows into groups of 1440 (minutes per day)
row_groups <- as.factor((seq_len(nrow(dat)) - 1) %/% 1440 + 1)

cat("Creating movement heatmap...\\n")
cat("Data dimensions:", dim(dat), "\\n")
cat("Row groups (days):", length(unique(row_groups)), "\\n")

ht_mt <- Heatmap(dat,
        name = "mt",
        col = colorRamp2(c(0, 1, 10), c("black", "white", "red")),
        cluster_rows = FALSE,
        cluster_columns = FALSE,
        row_names_gp = gpar(fontsize = 0),
        column_names_gp = gpar(fontsize = 0),
        column_split = monitor$meta.data$Group,
        row_split = row_groups)

# Save the heatmap
png(file=file.path(data_dir, "heatmap_mt.png"), width=8000, height=4000, res=300)
draw(ht_mt)
dev.off()

cat("Movement heatmap saved to:", file.path(data_dir, "heatmap_mt.png"), "\\n")
''')

In [ ]:
# Create Position (pn) Heatmap
r('''
# Heatmap of position data (pn)
dat <- as.matrix(monitor$assays$pn)
colnames(dat) <- colnames(monitor$assays$pn)

# Create a factor that divides the rows into groups of 1440
row_groups <- as.factor((seq_len(nrow(dat)) - 1) %/% 1440 + 1)

cat("Creating position heatmap...\\n")
cat("Position data range:", range(dat, na.rm = TRUE), "\\n")

ht_pn <- Heatmap(dat,
        name = "pn",
        col = colorRamp2(c(1, 8, 15), c("blue", "white", "red")),
        cluster_rows = FALSE,
        cluster_columns = FALSE,
        row_names_gp = gpar(fontsize = 0),
        column_names_gp = gpar(fontsize = 0),
        column_split = monitor$meta.data$Group,
        row_split = row_groups)

# Save the heatmap
png(file=file.path(data_dir, "heatmap_pn.png"), width=8000, height=4000, res=300)
draw(ht_pn)
dev.off()

cat("Position heatmap saved to:", file.path(data_dir, "heatmap_pn.png"), "\\n")
''')

In [ ]:
# Perform Sleep Analysis
r('''
# Sleep analysis - convert movement sequences to awake/sleep states
# Source the required functions
source("./src/convert_sequence.R")
source("./src/flip_binary.R")

cat("Performing sleep analysis...\\n")

# Convert movement data to awake states
monitor$assays$awake_mt <- convert_sequences(monitor$assays$mt)
cat("Awake data created - dimensions:", dim(monitor$assays$awake_mt), "\\n")

# Create sleep data by flipping awake states
monitor$assays$sleep_mt <- flip_binary(monitor$assays$awake_mt)
cat("Sleep data created - dimensions:", dim(monitor$assays$sleep_mt), "\\n")

# Show some statistics
awake_prop <- mean(monitor$assays$awake_mt, na.rm = TRUE)
sleep_prop <- mean(monitor$assays$sleep_mt, na.rm = TRUE)
cat("Average proportion awake:", round(awake_prop, 3), "\\n")
cat("Average proportion asleep:", round(sleep_prop, 3), "\\n")
''')

In [ ]:
# Create Sleep Heatmap
r('''
# Heatmap of sleep data
dat <- as.matrix(monitor$assays$sleep_mt)
colnames(dat) <- colnames(monitor$assays$sleep_mt)

# Create a factor that divides the rows into groups of 1440
row_groups <- as.factor((seq_len(nrow(dat)) - 1) %/% 1440 + 1)

cat("Creating sleep heatmap...\\n")
cat("Sleep data range:", range(dat, na.rm = TRUE), "\\n")

ht_sleep <- Heatmap(dat,
              name = "sleep_mt",
              col = colorRamp2(c(0,1), c("white", "red")),
              cluster_rows = FALSE,
              cluster_columns = FALSE,
              row_names_gp = gpar(fontsize = 0),
              column_names_gp = gpar(fontsize = 0),
              column_split = monitor$meta.data$Group,
              row_split = row_groups)

# Save the heatmap
png(file=file.path(data_dir, "heatmap_sleep.png"), width=8000, height=4000, res=300)
draw(ht_sleep)
dev.off()

cat("Sleep heatmap saved to:", file.path(data_dir, "heatmap_sleep.png"), "\\n")
''')

In [ ]:
# Calculate Position When Awake
r('''
# Only position when flies are awake
# Multiply position data by awake state to get position only when awake
cat("Calculating position when awake...\\n")

monitor$assays$pn_awake <- monitor$assays$pn * monitor$assays$awake_mt

cat("Position-awake data created - dimensions:", dim(monitor$assays$pn_awake), "\\n")

# Show some statistics
non_zero_prop <- mean(monitor$assays$pn_awake > 0, na.rm = TRUE)
cat("Proportion of non-zero position-awake values:", round(non_zero_prop, 3), "\\n")
cat("Position-awake data range:", range(monitor$assays$pn_awake, na.rm = TRUE), "\\n")
''')

In [ ]:
# Create Position-Awake Heatmap
r('''
# Heatmap of position when awake
dat <- as.matrix(monitor$assays$pn_awake)
colnames(dat) <- colnames(monitor$assays$pn_awake)

# Create a factor that divides the rows into groups of 1440
row_groups <- as.factor((seq_len(nrow(dat)) - 1) %/% 1440 + 1)

cat("Creating position-awake heatmap...\\n")

ht_pn_awake <- Heatmap(dat,
              name = "pn_awake",
              col = colorRamp2(c(0, 1, 15), c("white", "blue", "red")),
              cluster_rows = FALSE,
              cluster_columns = FALSE,
              row_names_gp = gpar(fontsize = 0),
              column_names_gp = gpar(fontsize = 0),
              column_split = monitor$meta.data$Group,
              row_split = row_groups)

# Save the heatmap
png(file=file.path(data_dir, "heatmap_pn_awake.png"), width=8000, height=4000, res=300)
draw(ht_pn_awake)
dev.off()

cat("Position-awake heatmap saved to:", file.path(data_dir, "heatmap_pn_awake.png"), "\\n")
''')

In [ ]:
# Generate Frequency Distributions and Bar Plots
r('''
# Source required functions for frequency analysis and plotting
source("./src/df_to_freq_dist.R")
source("./src/bar_plot.R")

cat("Generating frequency distributions and bar plots...\\n")

# Convert position-awake data to frequency distribution
pn_awake_freq <- df_to_freq_dist(monitor$assays$pn_awake)
colnames(pn_awake_freq) <- colnames(monitor$assays$pn_awake)
rownames(pn_awake_freq) <- c(1:15)

# Save frequency distribution for all data
write.csv(pn_awake_freq, file=file.path(data_dir, "pn_awake_freq_all.csv"))
cat("Frequency distribution saved to:", file.path(data_dir, "pn_awake_freq_all.csv"), "\\n")

# Prepare data for plotting
df_long <- prepare_data(monitor, pn_awake_freq, data_dir)
results <- calculate_avg_proportion(df_long)

# Create bar plot for all data
create_bar_plot(results$df_avg, results$df_cat1, file.path(data_dir, "barplot_pn1_awake_all.png"))
cat("Bar plot saved to:", file.path(data_dir, "barplot_pn1_awake_all.png"), "\\n")

cat("Overall frequency analysis completed!\\n")
''')

In [ ]:
# Process Daily Data Separately
r('''
# Process each day separately (days 1-5)
cat("Processing daily data separately...\\n")

for (day in 1:5) {
  cat("Processing day", day, "...\\n")
  
  # Calculate row indices for this day (1440 minutes per day)
  start_row <- (day - 1) * 1440 + 1
  end_row <- day * 1440
  
  # Extract data for this day
  day_data <- monitor$assays$pn_awake[start_row:end_row,]
  
  # Convert to frequency distribution
  pn_awake_freq <- df_to_freq_dist(day_data)
  colnames(pn_awake_freq) <- colnames(monitor$assays$pn_awake)
  rownames(pn_awake_freq) <- c(1:15)
  
  # Save frequency distribution for this day
  freq_file <- file.path(data_dir, paste0("pn_awake_freq_day", day, ".csv"))
  write.csv(pn_awake_freq, file=freq_file)
  cat("  Frequency distribution saved to:", freq_file, "\\n")
  
  # Prepare data and create bar plot for this day
  df_long <- prepare_data(monitor, pn_awake_freq, data_dir)
  results <- calculate_avg_proportion(df_long)
  
  plot_file <- file.path(data_dir, paste("barplot_pn1_awake_day", day, ".png"))
  create_bar_plot(results$df_avg, results$df_cat1, plot_file)
  cat("  Bar plot saved to:", plot_file, "\\n")
}

cat("\\nDaily analysis completed for all 5 days!\\n")
cat("\\n=== ANALYSIS COMPLETE ===\\n")
cat("All heatmaps, frequency distributions, and bar plots have been generated.\\n")
cat("Check the", data_dir, "directory for output files.\\n")
''')

## Analysis Summary

This notebook has completed a comprehensive analysis of fly behavior data, including:

### Generated Outputs:
1. **Heatmaps:**
   - `heatmap_mt.png` - Movement data visualization
   - `heatmap_pn.png` - Position data visualization  
   - `heatmap_sleep.png` - Sleep pattern visualization
   - `heatmap_pn_awake.png` - Position when awake visualization

2. **Frequency Distributions:**
   - `pn_awake_freq_all.csv` - Overall frequency distribution
   - `pn_awake_freq_day1.csv` through `pn_awake_freq_day5.csv` - Daily distributions

3. **Bar Plots:**
   - `barplot_pn1_awake_all.png` - Overall position frequency plot
   - `barplot_pn1_awake_day1.png` through `barplot_pn1_awake_day5.png` - Daily plots

### Key Analysis Steps:
- Data loading and preprocessing
- Metadata integration
- Time range selection and data merging
- Dead fly removal
- Sleep state analysis (awake/sleep conversion)
- Position tracking during awake periods
- Statistical analysis and visualization

All files have been saved to the `./test/` directory for further analysis and review.